In [1]:
import pandas as pd
import datetime

In [2]:
# 读取数据集
df1 = pd.read_excel('./实例文件.xlsx',sheet_name='data1')
df1

,occur_time,serveID,metric_name,maximum
0,2021-02-11 15:26:10,1-230,CpuUsage,63.85
1,2021-02-11 15:26:10,1-230,MemoryUsage,40.21
2,2021-02-11 15:26:10,2-459,CpuUsage,63.85
3,2021-02-11 15:26:10,2-459,MemoryUsage,40.21
4,2021-02-11 15:26:20,1-230,CpuUsage,52.76
5,2021-02-13 17:16:20,1-230,CpuUsage,95.32


In [3]:
df2 = pd.read_excel('./实例文件.xlsx',sheet_name='data2')
df2

,uid,occur_date,first_time
0,256,2020-12-30,2020-12-30 20:00:00
1,256,2020-12-12,2020-12-12 09:00:00
2,256,2020-12-08,2020-12-08 12:00:00
3,256,2020-12-01,2020-12-01 21:00:00
4,256,2020-11-20,2020-11-20 19:00:00
5,258,2020-12-31,2020-12-31 23:00:00
6,258,2020-12-18,2020-12-18 19:00:00
7,258,2020-12-08,2020-12-08 02:00:00
8,258,2021-01-01,2021-01-01 11:00:00
9,258,2020-10-20,2020-10-20 09:00:00


In [4]:
# 场景一
# 对所有服务器进行一次巡检，了解每台服务器在最近一周内 CPU 消耗最高使用情况，用于判断服务器资源是否足够支撑业务健康运行，为了过滤掉部分干扰，每台服务器需要去掉最高的千分之一数据
today = '2021-02-13'
begin_date = (pd.to_datetime(today) - datetime.timedelta(days=7)).strftime('%Y/%m/%d')
end_date = (pd.to_datetime(today) - datetime.timedelta(days=1)).strftime('%Y/%m/%d')
con1 = df1['occur_time'] >= begin_date
con2 = df1['occur_time'] <= end_date
df_cpu = df1[df1['metric_name'].str.contains('Cpu') & con1 & con2]
df_group = df_cpu.groupby(['serveID'])['maximum'].max().reset_index()
df_group['minus_1/1000'] = df_group['maximum'] * (1 - 0.001)
df_group

,serveID,maximum,minus_1/1000
0,1-230,63.85,63.78615
1,2-459,63.85,63.78615


In [5]:
# 场景二
# 分析玩家最后 4 个登录日每日首次上线时间段的偏好，便于某活动推广
# 取每个玩家最后 4 条的登录流水，玩家历史登录流水不足 4 条的也取出
# 取登录日首次上线时的小时，计算出每个玩家上线最多的时间段
# [0点，6点) ——> 凌晨
# [6点，12点) ——> 上午
# [12点，18点) ——> 下午
# [18点，24点) ——> 夜晚
df2['hour'] = df2['first_time'].dt.hour
df2_sort = df2.sort_values(by=['uid','occur_date'],ascending=True)
df2_sort['组内降序排名'] = df2_sort.groupby(['uid'])['occur_date'].rank(method='dense',ascending=False).astype(int)
df2_filiter = df2_sort[df2_sort['组内降序排名'] <= 4]
df2_filiter['时间段'] = pd.cut(df2_filiter['hour'],[0,6,12,18,24],labels=['凌晨','上午','下午','夜晚'])
df = df2_filiter.groupby(['uid','时间段'])['occur_date'].count().reset_index()
df['组内降序排名'] = df.groupby(['uid'])['occur_date'].rank(method='dense',ascending=False).astype(int)
df = df[df['组内降序排名'] <= 1]
df = df.rename(columns={'occur_date':'首登次数'}).drop(columns=['组内降序排名'])
df

D:\ProgramData\Anaconda\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


,uid,时间段,首登次数
1,256,上午,2
3,256,夜晚,2
7,258,夜晚,2
